In [ ]:
import glob
import os
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
import h5py
import yaml

from astroquery.mast import Catalogs

from thejoker import JokerSamples
from twoface.ext import FileLoader
from twoface.mass import asini, a2sini, stellar_radius, get_m2_min, mf

In [ ]:
cache_path = '../data/candidates/cache'

Catalogs we should query:
* Galex
* PS1
* WISE

In [ ]:
allstar = fits.getdata('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58158.fits')

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
candidates = sheet.load()

In [ ]:
omg = ['2M00444105+8351358',
       '2M04015112+5316431',
       '2M05121632+4558157',
       '2M05335134+2625329',
       '2M06185816+4002167']

In [ ]:
nsamples = 10000
for row in candidates[np.isin(candidates['APOGEE_ID'], omg)]:
    apogee_id = row['APOGEE_ID']
    star = allstar[allstar['APOGEE_ID'] == apogee_id]
    samples_fn = path.join(cache_path, '{0}-joker.hdf5'.format(apogee_id))
    with h5py.File(samples_fn) as f:
        samples = JokerSamples.from_hdf5(f)
    c = coord.SkyCoord(ra=row['RA']*u.deg, 
                   dec=row['DEC']*u.deg)
                        
    print('{0}:'.format(row['APOGEE_ID']))
    print('\t{0.ra.degree:.6f} {0.dec.degree:.6f}'.format(c))
    
    if star['LOGG'][0] < 0: continue
        
    logg = np.random.normal(star['LOGG'][0], star['LOGG_ERR'][0], size=nsamples)
    M1 = np.random.normal(row['M1'], 0.1, nsamples) * u.Msun # MAGIC NUMBER
    R1 = stellar_radius(logg, M1).to(u.au)
    
    mfs = mf(samples['P'], samples['K'], samples['e'])
    M2 = get_m2_min(np.repeat(M1[:, None], len(mfs), axis=1).ravel(),
                    np.repeat(mfs[None], len(M1), axis=0).ravel())
    M2 = M2.reshape(nsamples, len(samples))
    a2_min = a2sini(samples['P'][None], samples['e'][None], samples['K'][None], 
                    M1[:, None], M2)
    a_min = asini(samples['P'][None], samples['e'][None], samples['K'][None], 
                  M1[:, None], M2)
    q = M2 / M1[:, None]
    R_a = 0.49 * (q**(-2/3)) / (0.6 * q**(-2/3) + np.log(1 + q**(-1/3)))
    
    print('\tM1 = {:.2f}'.format(row['M1']*u.Msun))
    print('\tM2sini = {:.2f} +/- {:.2f}'.format(np.median(M2), np.std(M2)))
    print('\tR1 = {:.3f} +/- {:.3f}'.format(np.median(R1), np.std(R1)))
    print('\ta2sini = {:.2f} +/- {:.2f}'.format(np.median(a2_min), 
                                                np.std(a2_min)))
    print('\t[Fe/H] = {:.3f} +/- {:.3f}'.format(star['M_H'][0], star['M_H_ERR'][0]))
    print('\tRoche R/a = {:.2f}, R1/asini = {:.2f}'.format(np.median(R_a), np.median(R1[:, None] / a_min)))
    print('')

In [ ]:
from astropy.table import QTable
from thejoker import RVData
from twoface.samples_analysis import unimodal_P
def load_data_ecsv(fn):
    tbl = QTable.read(fn)
    data = RVData(t=tbl['time'],
                  rv=tbl['rv'],
                  stddev=tbl['rv_err'])
    return data, tbl['source']

for row in candidates:
    apogee_id = row['APOGEE_ID']
    samples_fn = path.join(cache_path, '{0}-joker.hdf5'.format(apogee_id))
    data_fn = path.join('../data/candidates/{0}.ecsv'.format(apogee_id))
    with h5py.File(samples_fn) as f:
        samples = JokerSamples.from_hdf5(f)
        
    data, _ = load_data_ecsv(data_fn)
    
    print(apogee_id, unimodal_P(samples, data))

In [ ]:
# from astroquery.mast import Catalogs
# mast_tbl = Catalogs.query_region("83.46395 26.425833", radius=10*u.arcsec)
# mast_tbl

In [ ]:
job = tap.launch_job('''SELECT TOP 100 * FROM "II/312/ais" WHERE 1=CONTAINS(POINT('ICRS',"II/312/ais".RAJ2000,"II/312/ais".DEJ2000), CIRCLE('ICRS', {0.ra.degree}, {0.dec.degree}, 10/3600.))'''.format(c))

In [ ]:
job.get_results()

In [ ]:
for c in all_c:
    mast_tbl = Catalogs.query_region(c, radius=10*u.arcsec)
    break

In [ ]:
Catalogs.q

In [ ]:
c.ra.degree, c.dec.degree

In [ ]:
mast_tbl